In [1]:
#import modules
import pandas as pd
import json
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os
import csv
from pandas.api.types import CategoricalDtype
low_memory = False

In [4]:
#read csv data from the CDC for CA
#I removed the cdc_ca_output_data.csv from the repo because it is too large to push to github
#it is now stored locally on my computer, which is why running this cell leads to an error message
csv_ca_cdc_path = "./Resources/CA_cases_by_county/cdc_ca_output_data.csv"
ca_covid = pd.read_csv(csv_ca_cdc_path, encoding="utf-8")
ca_covid = pd.DataFrame(ca_covid)
ca_covid = ca_covid.drop(columns=['res_state'])
ca_covid.head()

In [ ]:
#fill all null resident county values with 'small county' since the CDC does not include
#counties with tiny populations.

ca_covid['res_county'].fillna("SMALL COUNTY", inplace = True)
ca_covid['res_county'].unique()

In [ ]:
#read in Population Data from the Department of Finance (which the CDC uses)
#as well as partial/full vacciantion numbers
ca_pop_vac_path = './Resources/CA_county_population_and_vaccination_data.csv'
ca_pop_vac = pd.read_csv(ca_pop_vac_path)
ca_popvac_df = pd.DataFrame(ca_pop_vac)
ca_popvac_df.head()

In [ ]:
#make a list which counties are not included, though this may not be necessary for the final notebook.
#it is important to mention in the presentation, however.
print(len(ca_covid['res_county'].unique()))

cdc_included_counties = ca_covid['res_county'].unique()
cdc_included_counties = cdc_included_counties.tolist()

counties_list = ['Alameda', 'Alpine', 'Amador' ,'Butte' ,'Calaveras' ,'Colusa' ,'Contra Costa' ,'Del Norte'
                 ,'El Dorado' ,'Fresno' ,'Glenn' ,'Humboldt' ,'Imperial' ,'Inyo' ,'Kern' ,'Kings' ,'Lake' ,'Lassen',
                 'Los Angeles' ,'Madera' ,'Marin' ,'Mariposa' ,'Mendocino' ,'Merced' ,'Modoc' ,'Mono' ,'Monterey' ,'Napa', 
                 'Nevada' ,'Orange' ,'Placer' ,'Plumas' ,'Riverside' ,'Sacramento' ,'San Benito' ,'San Bernardino' ,'San Diego' ,
                 'San Francisco' ,'San Joaquin' ,'San Luis Obispo' ,'San Mateo' ,'Santa Barbara' ,'Santa Clara' ,'Santa Cruz' ,
                 'Shasta' ,'Sierra' ,'Siskiyou' ,'Solano' ,'Sonoma' ,'Stanislaus' ,'Sutter' ,'Tehama' ,'Trinity' ,'Tulare' ,'Tuolumne' ,
                 'Ventura' ,'Yolo' ,'Yuba']

small_county = []
#print(len(counties_list))
for county in counties_list:
    if county.upper() not in cdc_included_counties:
        small_county.append(county)
#print(small_county)

In [ ]:
#extract only the data for August 2020 on, was originally just for 2021
ca_cases_aug20_on = ca_covid.loc[(ca_covid['case_month'] == '2020-08') |(ca_covid['case_month'] == '2020-09') |
                            (ca_covid['case_month'] == '2020-10') |(ca_covid['case_month'] == '2020-11') |
                             (ca_covid['case_month'] == '2020-12') |
                              (ca_covid['case_month'] == '2021-01') | (ca_covid['case_month'] == '2021-02') | 
                              (ca_covid['case_month'] == '2021-03') | (ca_covid['case_month'] == '2021-04')  
                            | (ca_covid['case_month'] == '2021-05') | (ca_covid['case_month'] == '2021-06') 
                            | (ca_covid['case_month'] == '2021-07') ]                               

#extract just Dec 2020 on for group members to potentially use
ca_cases_2021 = ca_cases_aug20_on.copy()
ca_cases_2021 = ca_cases_2021.loc[(ca_cases_2021['case_month'] == '2020-12') |
                             (ca_cases_2021['case_month'] == '2021-01') | (ca_cases_2021['case_month'] == '2021-02') | 
                              (ca_cases_2021['case_month'] == '2021-03') | (ca_cases_2021['case_month'] == '2021-04')  
                            | (ca_cases_2021['case_month'] == '2021-05') | (ca_cases_2021['case_month'] == '2021-06') 
                           | (ca_cases_2021['case_month'] == '2021-07') ]  
##print(ca_cases_aug20_on.head())
#print(ca_cases_2021.head())

In [ ]:
#write out the modified dataframe
ca_cases_aug20_on.head()
ca_cases_aug20_on.to_csv('./Resources/CA_cases_by_county/ca_aug2020_on_case_data.csv',
                    index=False,header=True)

In [ ]:
#convert the case_month to month names so that it can merge with Tikaram
#use DatetimeIndex so that computer does not time out in for loop
ca_cases_aug20_on['month'] = pd.DatetimeIndex(ca_cases_aug20_on['case_month']).month_name()
ca_cases_2021['month'] = pd.DatetimeIndex(ca_cases_2021['case_month']).month_name()

In [ ]:
ca_cases_aug20_on.head(3)

In [ ]:
#do this again for August 2020 on
#the warning message is because Jupyter thinks I want the changes in ca_cases_aug20_on
#and ca_cases_2021 linked, though I do not
aug_on_month = ca_cases_aug20_on["month"]

month_order = CategoricalDtype(['August','September','October','November','December',
                                'January', 'February', 'March', 'April',
                                'May', 'June', 'July'], ordered=True)

ca_cases_aug20_on['month'] = ca_cases_aug20_on['month'].astype(month_order)

In [ ]:
#this will create and write out a df just for the months Tikaram can use.
grouped_by_2021_month = ca_cases_2021.groupby(['month','res_county']).size()


#here I convert the pandas series to a df and label the case counts
grouped_by_2021_month_df = grouped_by_2021_month.to_frame(name = 'cases')
grouped_by_2021_month_df.to_csv('./Resources/CA_cases_by_county/cases_by_county_and_month_2021.csv')

In [ ]:
#this took me a long time to figure out how to do, but it groups by month and county,
#while counting the occurences of each case. .size() makes it into a pandas series
#instead of a groupby object

grouped_by_month = ca_cases_aug20_on.groupby(['month','res_county']).size()
print(grouped_by_month)

#here I convert the pandas series to a df and label the case counts
grouped_by_month_df = grouped_by_month.to_frame(name = 'cases')


In [ ]:
#write the grouped/counted df to csv
grouped_by_month_df.to_csv('./Resources/CA_cases_by_county/cases_by_county_and_month.csv')
grouped_by_month_df.plot(kind='area')

In [ ]:
#since since using groupby followed by .sum() 
#is taking so long, make variables for the case count each month

counts = ca_cases_aug20_on['month'].value_counts()

aug = counts['August']
sept = counts['September']
octo = counts['October']
nov = counts['November']
dec = counts['December']
jan = counts['January']
feb = counts['February']
mar = counts['March']
apr = counts['April']
may = counts['May']
june = counts['June']
july = counts['July']
month_list = counts.tolist()
ordered_month_list = [151345,100714,123809,415153,1216142,873586,181288,82660,65037,37661,34899,71623]
#print(month_list)
print(counts)
#ca_cases_aug20_on.head(3)

In [ ]:
total_county_cases_df = ca_cases_aug20_on[['month','res_county']]

#make dataframes of total monthly cases and total cases by county
#for future reference, groupby coupled with .size() is a faster way
#to get the case count than .sum() it also makes it easy to convert a groupby object to df
county_total_cases_df = total_county_cases_df.groupby(by='res_county').size() 
county_total_cases_df = county_total_cases_df.to_frame(name = 'cases')

total_per_month_df = pd.DataFrame({
    "Months" : ['December','January', 'February', 'March', 'April',
                 'May', 'June', 'July'], 
    "Cases" : [dec,jan,feb,mar,apr,may,june,july]
})

#print(total_per_month_df)
total_per_month_df.to_csv('./Resources/CA_cases_by_county/total_monthly_cases.csv', index = False)
county_total_cases_df.to_csv('./Resources/CA_cases_by_county/total_county_cases.csv')

In [ ]:

age_counts = ca_cases_aug20_on['age_group'].value_counts()
print(age_counts)
case_status_counts = ca_cases_aug20_on['current_status'].value_counts()
#print(case_status_counts)
sex_counts = ca_cases_aug20_on['sex'].value_counts()
#print(sex_counts)
race_counts = ca_cases_aug20_on['race'].value_counts()
#print(race_counts)
ethnicity_counts = ca_cases_aug20_on['ethnicity'].value_counts()
#print(ethnicity_counts)
#county_counts = ca_cases_aug20_on['res_county'].value_counts()


In [ ]:
#for some reason this cell needs to be run manually, the graph will not appear
#by restarting and running all automatically and needs to be run twice for the
#x-axis tick label size to be adjusted

%matplotlib notebook
import matplotlib.pyplot as plt
#show how many cases there are per month
months_strings = ['August','September','October','November','December','January'
                 ,'February', 'March', 'April', 'May','June','July']
x_axis = months_strings
y_axis = ordered_month_list
#print(month_list)

plt.figure(figsize=(8.5,5))
graph = plt.bar(x_axis,y_axis)
#graph = plt.gcf()
plt.xlabel("Months", fontsize=12)
plt.ylabel("Case Numbers in Millions",fontsize=12)
plt.title('Cases by Month')
plt.rc('xtick', labelsize = 7)
plt.tight_layout
#graph = plt.set_size_inches(14,10)
plt.show()
plt.savefig('./Resources/CA_cases_by_county/Images/total_cases_by_month.png')




In [ ]:
%matplotlib notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 15]
age_group_cases_monthly_df.plot(kind='bar')
plt.figure(figsize=(20,5))
plt.show()
#plt.savefig('./Resources/CA_cases_by_county/Images/age_group_month_chart.png')



In [ ]:
#do pie chart for age
%matplotlib notebook
import matplotlib.pyplot as plt
labels = ['0-17 years', '18-49 years','50-64 years','65+ years'] 
age_cases = [455570,1916866,629910,342849] 
colors = ['red','blue','yellow','green']
plt.title('Percent of CA COVID Cases by Age Group')
plt.pie(age_cases, labels=labels, colors=colors,  autopct="%1.1f%%", shadow = True)
plt.show()

plt.savefig('./Resources/CA_cases_by_county/Images/pie_chart_ages.png')

In [ ]:
#make pie chart and dataframe for sex and cases
%matplotlib notebook
sex_df = ca_cases_aug20_on[['sex','month', 'res_county']]
sex_df.head(3)
labels = ['Female','Male','Unknown']
sex_cases = [1696862,1596324,47378]
colors = ['red','blue','yellow']
plt.pie(sex_cases,labels=labels,colors=colors,shadow=True,autopct="%1.1f%%")
plt.title('Percent of CA COVID Cases by Sex')
plt.savefig('./Resources/CA_cases_by_county/Images/pie_chart_sex.png')
#sex_counts = ca_cases_aug20_on['sex'].value_counts()
#print(sex_counts)


In [ ]:
#charts for known race/ethnicity and cases info. Not including unknown/null values.
#ethnicity is only Hispanic/non-Hispanic so I am including it here.
%matplotlib notebook
%matplotlib inline
plt.rcParams['figure.figsize'] = [15, 15]

race_df = ca_cases_aug20_on[['race','month', 'res_county']]
race_df.head(3)
labels = ['Native Hawaiian/Other Pacific Islander','White', 'Multiple/Other', 'Asian', 'Black', 'American Indian/Alaska Native', 
           'Hispanic/Latino']
race_ethnicity_cases = [12476,831768,278682,161342,112558,13803,488283]
plt.figure(figsize=(15,5))
plt.bar(labels,race_ethnicity_cases)
plt.tight_layout()
plt.xlabel('Race/Ethnicity')
plt.ylabel('Total Cases 08/20-07/21')
plt.title('CA Case Totals by Race/Ethnicity 08/20-07/2021')
plt.rc('xtick', labelsize = 10)

#race_counts = ca_cases_aug20_on['race'].value_counts()
#print(race_counts)
#ethnicity_counts = ca_cases_aug20_on['ethnicity'].value_counts()
#print(ethnicity_counts)
plt.savefig('./Resources/CA_cases_by_county/Images/bar_chart_race_ethnicity.png')
plt.show()

In [ ]:
#read in vaccine data, deaths data cleaned in my 'ca_covid_deaths_data' notebook
#and the 'cases_grouped_by_county_and_month.csv' (which is actually the grouped_by_month_df)
